Prototype of model explanation via LIME with help of extractive summary
---

---

In [1]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras

import time

tfds.disable_progress_bar()

In [2]:
now = time.strftime("%Y-%m-%d_%H:%M")

In [3]:
import logging
import sys

logging.basicConfig(
    level=logging.DEBUG, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(filename=f'../../data/logs/v1-{now}.log'),
        logging.StreamHandler(sys.stdout)
    ]
)

l = logging.getLogger('prototype')
l.critical("Logging LIME with new TF model")

[{<ipython-input-3-d59e77e64a5c>:14} CRITICAL - Logging LIME with new TF model


---

Model-related
---

In [4]:
def load_model():
    """
    Define a function that loads a model to be explained and returns its instance
    """
    
    return keras.models.load_model("../../raw-data/lstm-model-sigmoid")    

In [5]:
model = load_model()
l.info("Model loaded")

[{<ipython-input-5-b2e056f55d37>:2} INFO - Model loaded


In [6]:
model.predict(["hahahahahahahahahaha this is the most boring film I have ever seen"])

array([[0.0706619]], dtype=float32)

In [7]:
model.predict_proba(["hahahahahahahahahaha this is the funniest film I have ever seen"])
# Even though model has function `predict_proba`, it is not sufficient for LIME
# LIME expects this predict_proba function to return probability for each of the predicted classes

Instructions for updating:
Please use `model.predict()` instead.
[{deprecation.py:317} WARNING - From <ipython-input-7-598b0595b113>:1: Sequential.predict_proba (from tensorflow.python.keras.engine.sequential) is deprecated and will be removed after 2021-01-01.
Instructions for updating:
Please use `model.predict()` instead.


array([[0.948632]], dtype=float32)

---

Explanation
---

#### 1. Preparation

In [8]:
from lime import lime_text

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.summarizers.text_rank import TextRankSummarizer

import os

from functools import reduce

In [244]:
INPUT_DIR = "/home/tomasmizera/school/diploma/src/data/reviews"

LANGUAGE = "english"
SENTENCES_COUNT = 6
TOP_FEATURES_COUNT = 100

REVIEW_IX = 0
EXPL_IX = 1

In [10]:
summarizer = TextRankSummarizer(Stemmer(LANGUAGE))
summarizer.stop_words = get_stop_words('english')

In [11]:
explanator = lime_text.LimeTextExplainer(class_names=['positive', 'negative'])

---

### 2. Execution

In [12]:
l.info("Starting an algorithm")

[{<ipython-input-12-9e84a1d64b6b>:1} INFO - Starting an algorithm


In [13]:
# define a decorator to log execusion time
# inspired by https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d

def timeit(method):
    def timed(*args, **kw):
        timed.calls += 1
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        timed.time_taken += (te - ts) * 1000
        return result
    timed.calls = 0
    timed.time_taken = 0
    return timed

In [35]:
# @timeit # for LIME it is called once and for this model takes around 150ms with logging
def _predict_proba(_input):
    """
    Define a function that accepts array of instances and returns a probability for each class 
    _input - 1d array of instances
    Returns 2d array of [num of instances] x [num of classes] with probabilities
    """
    prediction = model.predict( _input )
    
    return np.append(prediction, 1 - prediction, axis=1)

In [37]:
def _explain_instance(_file, _explanator):
    explanation = _explanator.explain_instance(_file, _predict_proba, num_features=TOP_FEATURES_COUNT)
#     l.info('_predict_proba took:  %2.2f ms' % \
#                   (_predict_proba.time_taken))
#     l.info(f'_predict_proba called {_predict_proba.calls} times')
#     _predict_proba.calls = 0
#     _predict_proba.time_taken = 0
    
    return explanation

**Some examples of perturbed text**: \[ ..., ' great hm', ' great ', ' great hm', '  hm', '  ', '  hm', '  ', '  ', '  ', '  ', 'not great ', '  ', '  hm', '  ', '  ', 'not great ', '  ', ' great ', ' great ', '  hm', '  ', ' great ', 'not  hm', ' great hm', ' great ', ' great hm', '  hm', 'not  ', '  hm', 'not  ', ' great hm', 'not great ', ' great ', '  ', '  ', '  hm', '  ', '  ', '  ', 'not  hm', 'not  ', 'not great ', 'not  hm', 'not  hm', 'not great ', 'not  hm', '  ', ' great ', '  ', '  hm', 'not  hm', '  ', '  ', ' great ', '  ', '  ', ' great ', 'not  ', 'not  hm', ' great ', 'not  ', 'not  ', 'not  hm', 'not  ', '  ', 'not great ', '  hm', ' great hm', '  hm', '  ', '  ', 'not  hm', '  hm', 'not  ', '  ', ' great hm', 'not  ', ' great hm', 'not  hm', 'not  ', 'not  hm', '  ', ' great ', '  hm', ' great ', 'not  hm', 'not  ', 'not  ', ' great hm', 'not  hm', '  hm', '  hm', ' great ', '  ', 'not great ', '  hm', 'not great ', '  ', '  ', 'not  hm', 'not great ', '  ', '  ', ' great hm', 'not  hm', 'not  hm', ' great ', ' great hm', '  ', 'not  hm', ' great ', '  hm', ' great ', 'not great ', '  ', '  ', ' great hm', ' great hm', '  ', ' great ', 'not  hm', ' great ', 'not  ', ' great ', 'not great ', ' great ', 'not  ', 'not great ', '  ', '  ', '  ', 'not  ', ' great hm', ' great hm', '  ', '  ', 'not great ', '  ', '  ', 'not  ', ' great ', ' great ', 'not great ', '  ', '  ', '  ', '  ', '  ', '  ', '  hm', '  hm', 'not  ', 'not  hm', 'not  ', '  ', ' great ', '  hm', ' great hm', '  ', '  ', '  ', ' great hm', '  ', '  ', 'not  hm', '  ', ' great hm', ' great hm', ' great ', '  ', '  ', 'not  ', '  ', 'not  ', ' great hm', 'not great ', ' great hm', 'not  hm', 'not great ', 'not  ', 'not great ', '  ', 'not great ', '  hm', 'not  ', ' great ', '  ', '  ', ' great ', '  hm', 'not  ', 'not  ', 'not  ', '  ', ' great hm', ' great hm', ' great ', ' great ', 'not great ', ' great ', '  ', 'not great ', 'not great ', '  ', 'not  ', ' great ', ' great ', '  ', '  ', ' great ', 'not  ', ' great ', '  ', '  ', ' great ', 'not  hm', 'not  ', '  ', 'not great ', '  ', '  ', '  ', 'not great ', 'not great ', '  ', '  hm', '  hm', 'not  hm', 'not great ', ' great ', 'not  ', '  ', 'not  hm', 'not great ', 'not  ', 'not great ', 'not  hm', 'not  ', 'not  hm', ' great hm', ' great ', '  hm', '  ', '  hm', '  ', 'not  ', ' great ', '  ', '  hm', 'not  hm', 'not great ', '  ', '  ', ' great ', '  ', '  ', 'not  ', 'not  ', ' great ', '  ', 'not  hm', '  ', ' great hm', '  ', '  ', '  ', ' great ', ' great ', ' great ', ' great ', ' great ', '  hm', 'not  ', ' great ', 'not  hm', '  ', ' great hm', 'not great ', '  hm', '  hm', 'not  hm', 'not  ', 'not great ', '  ', '  ', '  ', 'not  ', '  ', ' great ', '  ', 'not great ', '  ', 'not  ', '  ', '  hm', ' great ', '  ', 'not  ', ' great hm', '  ', ' great ', 'not great ', '  ']

In [16]:
def input_from_files(path_to_files):
    """
    Loads all readable files in path_to_files directory
    Returns np.array with each files content as a separate element
    """
    
    def _read_text_file(filepath):
        with open(filepath, 'r') as f:
            return reduce(lambda a, b: a + b, f.readlines())
    
    files_it = os.scandir(path_to_files)
    files_contents = {}
    
    for file in files_it:
        if file.is_file(): 
            files_contents[file.name] = _read_text_file(file.path)
        
    return files_contents

In [17]:
npInput = input_from_files(INPUT_DIR)

In [33]:
def _summarize_doc_custom(_summarizer, _instance, _explanation):
    """
    Returns summary with altered weights based on explanation
    _summarizer - summy summarizer instance
    _instance - instance content string
    _explanation - LIME explanation
    """
    
    def _create_weight_graph(_summarizer, _instance_doc):
        return _summarizer.rate_sentences(_instance_doc)
    
    def _count_factor(_sentence, _explanation_words_weight) -> float: # returns boosting factor for sentence
        factor = 1.0
        exp_words = list(map(lambda x: x[0], _explanation_words_weight))
        for word in _sentence.words:
            if word in exp_words:
                factor += abs(_explanation_words_weight[exp_words.index(word)][1]) # TODO: apply factor scaling rather HERE than when returning! 
        return factor, 10 ** factor # factor * 3 if factor != 1 else factor # TODO: Parameter tuning for factor scale
    
    parser = PlaintextParser.from_string(_instance, Tokenizer(LANGUAGE))
    graph = _create_weight_graph(_summarizer, parser.document)
    
    for sentence in graph.keys():
        basic, factor = _count_factor(sentence, _explanation.as_list())
#         l.info("IX: Sentence " + str(sentence)[:20] + " previous weight: " + str(graph[sentence]) + " bf: " + str(basic) + " cf: " + str(factor) + " new weight: " + str(graph[sentence] * factor))
        graph[sentence] = graph[sentence] * factor 
        
    resulting_summary = _summarizer._get_best_sentences(parser.document.sentences, SENTENCES_COUNT, graph)
    
    return resulting_summary

In [19]:
def _summary_to_string(_summary):
    if len(_summary) <= 0:
        return ""
    
    summary_str = str(_summary[0])
    i = 1
    
    while(i < len(_summary)):
        summary_str += ' ' + str(_summary[i])
        i += 1
        
    return summary_str

In [20]:
def create_explanation_summaries(_instance_map, _explanator, _summarizer):
    """
    Returns summaries for all input elements
    _instance_map - map containing instance name and its content
    _explanator - LIME explanator instance
    _summarizer - summy summarizer instance
    """
    
    summaries = {}
    
    for instance in _instance_map.keys():
        explanation = _explain_instance(_instance_map[instance], _explanator)
        summary = _summarize_doc_custom(_summarizer, _instance_map[instance], explanation)
        summaries[instance] = (_summary_to_string(summary), explanation.as_list())

    return summaries

In [21]:
def create_simple_summaries(_instance_map, _summarizer):
    """
    Returns summaries for all input instances
    _instance_map - map containing instance name and its content
    _summarizer - summy summarizer instance
    """
    
    summaries = {}
    
    for instance in _instance_map.keys():
        _parser = PlaintextParser.from_string(_instance_map[instance], Tokenizer(LANGUAGE))
        summaries[instance] = _summary_to_string(_summarizer(_parser.document, SENTENCES_COUNT))
    
    return summaries

In [245]:
explanation_sums = create_explanation_summaries(npInput, explanator, summarizer);

In [29]:
simple_sums = create_simple_summaries(npInput, summarizer);

Summary visualization
---

---

In [265]:
import re

def highlight_summary(_summary):
    """
    TBD: all possible visualization options
    TODO: choose correct colors
    """
    colors = {}
    colors['blue'] = '54,151,186'
    colors['green'] = '0,127,0'
    
    start_highlight_tag = lambda col, a: f'<mark style="background-color:rgba({colors[col]},{a})">'
    end_highlight_tag = '</mark>'
    
    raw_text = _summary[0]
    important_words_weights = _summary[1]
    important_words = list(map(lambda x: x[0], important_words_weights))
    maxv = abs(important_words_weights[0][1])
    minv = abs(important_words_weights[-1][1]) # Here take abs for alpha calculations
    
    # TODO: maybe ignore words that has way too low weight?
    
    for ix, word in enumerate(important_words):
        # normalize alpha to 0, 1
        wx = important_words_weights[ix][1] 
        col = "blue" if wx >= 0 else "green"
        alpha = (abs(wx) - minv) / (maxv - minv) # consider weights as positive due to alpha
        pattern = r'(?<![><(=")\/])\b(' + word + r')\b(?!(:rgba)|(="back))'
        # https://regex101.com/r/nNu7Rs/1
        raw_text = re.sub(pattern, start_highlight_tag(col, alpha) + word + end_highlight_tag, raw_text, flags=re.I)
        
        
    display(HTML(raw_text))

In [266]:
highlight_summary(explanation_sums['review-top.txt'])

In [127]:
simple_sums['review-low.txt-test']

'One man’s determination to keep his life completely old-school proves utterly ruinous in “Hunter Hunter,” a movie written and directed by Shawn Linden. Devon Sawa plays Joe Mersault, a trapper whose nuclear family—wife-of-saintly-patience Anne (Camille Sullivan) and eager if occasionally queasy student of the traditional ways tween daughter Renée (Summer H. Howell)—live off the land in a cabin in some deep Northern woods (the movie was largely shot in Manitoba).'

TODO:
- [x] find a good pytorch/tf LSTM text classification model ~ maybe check datasets in LIME paper
- [x] create predict_proba based on the type of the framework
- [ ] predict on created summaries ~ automatically -> (possible: save summaries to files and then load and pass them just as normal instance) 
- [ ] refactor process to not store everything in RAM, rather put intermediate results to files
- [x] highlighting of important words from any summary (maybe save both, str summary and Sentence type summary - from sumy)
- [ ] extract it to separate script
- [ ] add better logging (more logs in this version)
- [ ] build and test quantitative experiment pipeline
- [ ] maybe find better dataset (longer texts) for data and train another model for it
- [ ] run quantitative experiment on all instances
- [ ] pick several (~6) explanations for user-study